### Pick a document

Can be CSV, PDF, TXT, etc. or a website

#### PDF

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

pdf_path = "context/Weller et al. - 2025 - On the Theoretical Limitations of Embedding-Based Retrieval.pdf"

loader = PyPDFLoader(
    file_path=pdf_path,
    mode="single"
)
docs = loader.load()

print(f"Loaded PDF as {len(docs):d} document(s)")
print("-"*50)
print("Metadata:")
for k, v in docs[0].metadata.items():
    print(f"{k}: {v}")
print("-"*50)
print("Page content (preview):")
print(docs[0].page_content[:500] + "...")

#### Wikipedia

In [1]:
from langchain_community.document_loaders import WikipediaLoader

loader = WikipediaLoader(
    query="What is the capital of France?",
    load_max_docs=10
)
docs = loader.load()

print(f"Loaded {len(docs):d} document(s)")
print("-"*50)
for doc in docs:
    print(doc.metadata)

Loaded 10 document(s)
--------------------------------------------------
{'title': 'Closed-ended question', 'summary': 'A closed-ended question is any question for which a researcher provides research participants with options from which to choose a response. Closed-ended questions are sometimes phrased as a statement that requires a response.\nA closed-ended question contrasts with an open-ended question, which cannot easily be answered with specific information.\n\n', 'source': 'https://en.wikipedia.org/wiki/Closed-ended_question'}
{'title': 'Capital punishment in the United States', 'summary': "In the United States, capital punishment (also known as the death penalty) is a legal penalty in 27 states (of which two, Oregon and Wyoming, have no inmates sentenced to death), throughout the country at the federal level, and in American Samoa. It is also a legal penalty for some military offenses. Capital punishment has been abolished in the other 23 states and in the federal capital, Wash

#### Custom Document

In [ ]:
from langchain_core.documents import Document

page_content = "Hello, world!"

docs = []
doc = Document(
    page_content=page_content,
    metadata={
        "source": "My custom document",
        "title": "Title of my custom document"
    }
)
docs.append(doc)

print("-"*50)
print("Metadata:")
for k, v in docs[0].metadata.items():
    print(f"{k}: {v}")
print("-"*50)
print("Page content (preview):")
print(docs[0].page_content if len(docs[0].page_content) < 500 else docs[0].page_content[:500] + "...")

#### Website

Caveat: Different websites have different structures and require different parsing strategies.

In [ ]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
import os
import re
os.environ['USER_AGENT'] = ("Demo")

"""
loader = WebBaseLoader(
    web_paths=("https://www.pff.com/news/nfl-scores-and-recaps-for-every-week-3-game",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer("article")
    ),
)
"""

loader = WebBaseLoader(
    web_paths=("https://www.cbssports.com/nfl/news/nfl-week-3-grades-scores-results-highlights-browns-packers-vikings-bengals/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer("article")
    ),
)

docs = loader.load()

# Clean and condense page content: strip, remove blank lines, trim lines, and join into a single line
docs[0].page_content = " ".join(
    line.strip() for line in docs[0].page_content.strip().splitlines() if line.strip()
)

# Replace sequences of two or more spaces with a single space
docs[0].page_content = re.sub(r'\s{2,}', ' ', docs[0].page_content)

print(f"Loaded Website as {len(docs):d} document(s)")
print("-"*50)
print("Metadata:")
for k, v in docs[0].metadata.items():
    print(f"{k}: {v}")
print("-"*50)
print("Page content (preview):")
print(docs[0].page_content if len(docs[0].page_content) < 10000 else docs[0].page_content[:10000] + "...")

### Chunking

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Split document into chunks for vector storage
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    add_start_index=True,
)
doc_chunks = text_splitter.split_documents(docs)

print(f"Document split into {len(doc_chunks):d} chunks.")

# Show first few chunks for verification
for index, chunk in enumerate(doc_chunks[:3]):
    print(f"\n\nChunk {index+1}")
    print("-"*50)
    print("-"*50)
    print("Metadata:")
    for k, v in chunk.metadata.items():
        print(f"{k}: {v}")
    print("-"*50)
    print("Page content:")
    print(chunk.page_content if len(chunk.page_content) < 500 else chunk.page_content[:500] + "...")

### Embed documents

In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_huggingface import HuggingFaceEmbeddings

In [ ]:
#embedding_model_name = "intfloat/multilingual-e5-base"
#embedding_model_name = "Qwen/Qwen2.5-0.5B"
#embedding_model_name = "Qwen/Qwen3-Embedding-0.6B"
embedding_model_name = "Qwen/Qwen3-Embedding-4B"
#embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"

embedding_function = HuggingFaceEmbeddings(
    model_name=embedding_model_name,
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},
)
vector_store = InMemoryVectorStore(embedding=embedding_function)

In [ ]:
# Add documents to vector store
document_chunk_ids = vector_store.add_documents(documents=doc_chunks)

print(f"Added {len(document_chunk_ids):d} documents to the vector store")

In [ ]:
# Inspect vector store
n_chunks = 5
for index, (id, doc) in enumerate(vector_store.store.items()):
    if index < n_chunks:
        print("\n"+"-"*100)
        print(f"Chunk {index+1}\n")
        print(f"id: {id}")
        print(f"vector (length: {len(doc['vector'])}): {doc['vector']}")
        print(f"metadata: {doc['metadata']}")
        print(f"text:\n{doc['text'][:100]}")
    else:
        break

### Test vector store

In [ ]:
# Test vector store with a sample query
query = "Who was the Vikings' starting quarterback in week 3?"

top_k = 5
similar_document_chunks = vector_store.similarity_search_with_score(query, k=top_k)

print(f"List of {len(similar_document_chunks):d} most similar document chunks for query: '{query:s}'")

for i, (doc, score) in enumerate(similar_document_chunks):
    if i < top_k:
        print("\n" + "-"*50)
        print(f"Result {i+1} (Similarity Score: {score:.4f})")
        print(f"\tid: {doc.id}")
        print(f"\tmetadata: {doc.metadata}")
        print(f"\tpage content: {doc.page_content if len(doc.page_content) < 300 else doc.page_content[:300] + '...'}")
    else:
        break

### Set up RAG query

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# Create prompt template for RAG system
chat_prompt = ChatPromptTemplate.from_messages([
    ("system",
    """
    You are an AI assistant that answers questions based on provided context documents. 
    """
    ),
    ("human",
    """
    Answer the question based on the context.

    CRITICAL RULES:
    - Answer concisely
    - Use information from the provided context
    - If the context doesn't contain enough information, state this clearly
    - Cite specific details from the context when possible

    CONTEXT:
    {context}

    QUESTION: {input}

    ANSWER:
    """
    )
])

### Load LLM

In [ ]:
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name = "google/gemma-3-1b-it"
print(f"Loading llm <{model_name:s}>")

# Load llm
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Create text generation pipeline
text_generation_pipeline = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=250,    # Limit answer length for concise RAG responses
    temperature=0.2,       # Low randomness, mostly deterministic
    top_p=0.95,            # Sample from top 95% probable tokens
    repetition_penalty=1.2 # Penalize repeated content to improve answer quality
)

# Wrap pipeline for LangChain
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

### The stuff documents chain

Takes your dummy_docs (like Peter's first name + Peter's last name)
Smooshes them all together into one big text

Uses your chat_prompt (the template that says "Hey AI, here's some context...")
Fills in the template with the smooshed-together documents

Sends everything to the llm (your AI friend)
Gets back a nice, smart answer

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain

combine_docs_chain = create_stuff_documents_chain(llm, chat_prompt)

retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k": 5,
        "score_threshold": 0.7,
    }
)

dummy_docs = [
    Document(
    page_content="My first name is Peter.",
    metadata={"doc-nr.": "1"}
    ),
    Document(
    page_content="My last name is Parker.",
    metadata={"doc-nr.": "2"}
    )
]

In [ ]:
response = combine_docs_chain.invoke({"context": dummy_docs, "input": "What is the person's full name?"})
print(f"Response:")
print("-"*50)
print(f"{response:s}")

### Retrieval chain

We don't want ALL the context, just the most relevant parts

So we use the retriever to get the most relevant parts
Then we use the stuff documents chain to answer the question

In [ ]:
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

response = retrieval_chain.invoke({"input": "What is this document about?"})

In [ ]:
print("ANSWER")
print("-"*50)
print(f"{response['answer']:s}")
print("\n\n")

print("CONTEXT")
print("-"*50)
for i, doc in enumerate(response['context']):
    print(f"Document {i+1}:")
    for key, value in doc.metadata.items():
        print(f"\t{key}: {value}")
    print(f"\tPage content: {doc.page_content if len(doc.page_content) < 300 else doc.page_content[:300] + '...'}")
    print("\n")
print("\n\n")

### Compare LLM vs RAG

In [ ]:
question = "Who was the Vikings' starting quarterback in week 3?"
print(f"Question: {question}")
print("\n")

print("LLM without context:")
print("-"*50)
llm_response = llm.invoke(question)
print(f"Answer: {llm_response}")
print("\n\n")

print("RAG system with context:")
print("-"*50)
rag_response = retrieval_chain.invoke({"input": question})
print(f"Answer: {rag_response['answer']}")

print("Context:")
for i, doc in enumerate(rag_response['context']):
    print(f"Document {i+1}:")
    for key, value in doc.metadata.items():
        print(f"\t{key}: {value}")
    print(f"\tPage content: {doc.page_content if len(doc.page_content) < 300 else doc.page_content[:300] + '...'}")
    print("\n")
print("\n\n")